# Netflix Text Analytics

<hr>

### Load Data From Table (Pyarrow Dataset)

```text
NETFLIX_REVIEWS
├── review_rating=0
│   └── 2418bd2b279a4829b6f69d61165509fd-0.parquet
├── review_rating=1
│   └── 2418bd2b279a4829b6f69d61165509fd-0.parquet
├── review_rating=2
│   └── 2418bd2b279a4829b6f69d61165509fd-0.parquet
├── review_rating=3
│   └── 2418bd2b279a4829b6f69d61165509fd-0.parquet
├── review_rating=4
│   └── 2418bd2b279a4829b6f69d61165509fd-0.parquet
└── review_rating=5
    └── 2418bd2b279a4829b6f69d61165509fd-0.parquet

```


```python
from pathlib import Path
import pyarrow.dataset as ds

# Specify the directory containing Parquet files
directory_path: Path = Path("../../../../data/NETFLIX_REVIEWS")

# Use PyArrow to read the partitioned Parquet dataset
table = ds.dataset(directory_path)

# Convert the PyArrow table to a Pandas DataFrame
data: pl.DataFrame = pl.scan_pyarrow_dataset(table).collect()

# Display the DataFrame
print(data)
```

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, TypeAlias, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Load Data

In [2]:
from pathlib import Path

# Specify the directory containing Parquet files
directory_path: Path = Path("../../../../data/NETFLIX_REVIEWS.parquet")

# Convert the PyArrow table to a Pandas DataFrame
data: pl.DataFrame = pl.read_parquet(
    directory_path,
).drop(columns=[""])

# Display the DataFrame
print(data)

shape: (1_531_126, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ review_id  ┆ pseudo_aut ┆ author_nam ┆ review_te ┆ review_ra ┆ review_li ┆ author_ap ┆ review_ti │
│ ---        ┆ hor_id     ┆ e          ┆ xt        ┆ ting      ┆ kes       ┆ p_version ┆ mestamp   │
│ str        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆ str        ┆ str        ┆ str       ┆ i64       ┆ i64       ┆ str       ┆ str       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7e73f80e-a ┆ 1526185539 ┆ A Google   ┆ Works     ┆ 5         ┆ 1         ┆ 1.2.0     ┆ 2011-05-1 │
│ 8fd-4ff3-b ┆ 7701969374 ┆ user       ┆ great on  ┆           ┆           ┆ build 819 ┆ 2         │
│ 09b-502f0a ┆ 2          ┆            ┆ my Evo!   ┆           ┆           ┆ 145-1.2.0 ┆ 18:50:37  │
│ d0…        ┆            ┆            ┆ Glad      ┆           ┆           ┆ -102      ┆           │
│            ┆            ┆            ┆ andr…     ┆           ┆           ┆           ┆           │
│ dab55eca-c ┆ 2343829428 ┆ A Google   ┆ Works     ┆ 5         ┆ 1         ┆ 1.2.0     ┆ 2011-05-1 │
│ 2a0-45a8-9 ┆ 6543707166 ┆ user       ┆ great on  ┆           ┆           ┆ build 819 ┆ 2         │
│ 3e3-9860c1 ┆ 7          ┆            ┆ HTC incre ┆           ┆           ┆ 145-1.2.0 ┆ 18:50:52  │
│ c5…        ┆            ┆            ┆ dible. C… ┆           ┆           ┆ -102      ┆           │
│ a3b8fa06-8 ┆ 1744736046 ┆ A Google   ┆ Works     ┆ 5         ┆ 0         ┆ 1.5.2     ┆ 2011-05-1 │
│ b8f-4f2f-a ┆ 0835879636 ┆ user       ┆ great on  ┆           ┆           ┆ build 389 ┆ 2         │
│ 1fa-fd37c4 ┆ 8          ┆            ┆ nexus s   ┆           ┆           ┆           ┆ 18:55:14  │
│ cb…        ┆            ┆            ┆           ┆           ┆           ┆           ┆           │
│ 837fdfa5-6 ┆ 2865934532 ┆ A Google   ┆ Working   ┆ 5         ┆ 0         ┆ 1.2.1     ┆ 2011-05-1 │
│ 06d-4cec-9 ┆ 1905488026 ┆ user       ┆ perfect   ┆           ┆           ┆ build 843 ┆ 2         │
│ e9a-e4a83d ┆ 9          ┆            ┆ for me on ┆           ┆           ┆ 839-1.2.0 ┆ 19:31:46  │
│ ad…        ┆            ┆            ┆ EVO, r…   ┆           ┆           ┆ -30       ┆           │
│ …          ┆ …          ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         ┆ …         │
│ afe340b9-6 ┆ 2175850666 ┆ Ma******** ┆ What more ┆ 2         ┆ 0         ┆ 8.94.0    ┆ 2023-11-1 │
│ 8df-4df9-8 ┆ 9482615615 ┆ ***ey      ┆ do you    ┆           ┆           ┆ build 10  ┆ 5         │
│ a86-7e9304 ┆ 9          ┆            ┆ want from ┆           ┆           ┆ 50546     ┆ 22:44:59  │
│ e1…        ┆            ┆            ┆ me tf…    ┆           ┆           ┆           ┆           │
│ 3015ab73-7 ┆ 2683859418 ┆ Em******** ┆ I will    ┆ 5         ┆ 0         ┆           ┆ 2023-11-1 │
│ 5e8-4f17-8 ┆ 1134330166 ┆ ****ey     ┆ love this ┆           ┆           ┆           ┆ 5         │
│ 377-4757ab ┆ 6          ┆            ┆ app       ┆           ┆           ┆           ┆ 22:45:05  │
│ bb…        ┆            ┆            ┆           ┆           ┆           ┆           ┆           │
│ 25b4b68e-a ┆ 2599939226 ┆ ****       ┆ The       ┆ 2         ┆ 0         ┆ 8.94.0    ┆ 2023-11-1 │
│ 432-4f21-b ┆ 2285477805 ┆            ┆ content   ┆           ┆           ┆ build 10  ┆ 5         │
│ f1c-68835f ┆ 8          ┆            ┆ is great  ┆           ┆           ┆ 50546     ┆ 22:48:54  │
│ 88…        ┆            ┆            ┆ but they  ┆           ┆           ┆           ┆           │
│            ┆            ┆            ┆ ke…       ┆           ┆           ┆           ┆           │
│ 7c8e755c-0 ┆ 5279250276 ┆ Ra******** ┆ I promise ┆ 1         ┆ 1         ┆ 8.94.0    ┆ 2023-11-1 │
│ 505-4e0c-a ┆ 7513192948 ┆ ****on     ┆ you on    ┆           ┆           ┆ build 10  ┆ 5         │
│ c89-719f8f ┆ 6          ┆            

<hr><br>

### Descriptive Statistics

- Distribution of rating.
- Number of unique authors.
- 5-summary stats of review ratings.
- Percentage of rating >= 3 and rating < 3
  

In [3]:
data.describe()

describe,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
str,str,str,str,str,f64,f64,str,str
"""count""","""1531126""","""1531126""","""1531126""","""1531126""",1.531126e6,1.531126e6,"""1531126""","""1531126"""
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0,"""0""","""0"""
"""mean""",null,null,null,null,3.91526,1.323521,null,null
"""std""",null,null,null,null,1.577103,32.524688,null,null
"""min""","""00000ca0-ac53-…","""10000001724899…","""!M***********e…","""""",0.0,0.0,"""""","""2011-05-12 18:…"
"""25%""",null,null,null,null,3.0,0.0,null,null
"""50%""",null,null,null,null,5.0,0.0,null,null
"""75%""",null,null,null,null,5.0,0.0,null,null
"""max""","""fffffb20-d555-…","""99999937773079…","""﴾G*********1﴿""","""🫶🏽""",5.0,8231.0,"""8.95.0 build 2…","""2023-11-15 22:…"


In [4]:
# Unique authors
data.select(pl.col("author_name").value_counts())

author_name
struct[2]
"{""gr*************el"",1}"
"{""Er********ka"",2}"
"{""va**********on"",1}"
"{""Az*****zu"",2}"
"{""Ha***********************ah"",1}"
"{""FI*********su"",1}"
"{""Pe*******mi"",1}"
"{""Ge************me"",1}"
"{""Mi*********ju"",1}"


In [5]:
# Unique review rating
data.select(pl.col("review_rating").value_counts(sort=True))

review_rating
struct[2]
"{5,935781}"
"{1,282021}"
"{4,156718}"
"{3,93793}"
"{2,62790}"
"{0,23}"


In [6]:
# Drop review rating of 0
data = data.filter(pl.col("review_rating").ne(0))

# Unique review rating
data.select(pl.col("review_rating").value_counts(sort=True))

review_rating
struct[2]
"{5,935781}"
"{1,282021}"
"{4,156718}"
"{3,93793}"
"{2,62790}"


In [7]:
# Percentage of review rating >=3
data.filter(pl.col("review_rating").gt(2)).select(pl.col("review_rating")).shape[
    0
] / data.shape[0]

0.7747956865083537

In [8]:
# Percentage of review rating <3
data.filter(pl.col("review_rating").lt(3)).select(pl.col("review_rating")).shape[
    0
] / data.shape[0]

0.2252043134916462

In [9]:
# Unique author_app_version
print(data.select(pl.col("author_app_version").n_unique()))
data.select(pl.col("author_app_version").value_counts())

shape: (1, 1)
┌────────────────────┐
│ author_app_version │
│ ---                │
│ u32                │
╞════════════════════╡
│ 720                │
└────────────────────┘

author_app_version
struct[2]
"{""5.8.0 build 24644"",16}"
"{""5.11.0 build 25361"",1}"
"{""8.10.0 build 10 40088"",1}"
"{""8.22.1 build 22 40193"",1934}"
"{""7.7.0 build 20 34181"",2160}"
"{""7.110.0 build 6 35520"",41}"
"{""8.67.1 build 11 50414"",955}"
"{""5.13.1 build 25871"",1460}"
"{""6.7.0 build 28708"",2}"


In [10]:
data.select(pl.col("author_app_version"))

author_app_version
str
"""1.2.0 build 81…"
"""1.2.0 build 81…"
"""1.5.2 build 38…"
"""1.2.1 build 84…"
"""1.5.2 build 38…"
"""1.4.1 build 26…"
"""1.5.2 build 38…"
"""1.2.0 build 81…"
"""1.2.0 build 81…"


In [11]:
pattern: str = r"\d{1,4}\.\d{1,4}\.\d{1,4}\s?\w+"

data = data.with_columns(
    pl.col("author_app_version").str.extract(pattern, 0).alias("cleaned_app_version")
)

In [12]:
df: pl.DataFrame = (
    data.group_by(pl.col("cleaned_app_version"))
    .agg(pl.mean("review_rating"))
    .sort(by="cleaned_app_version", descending=False)
)
df.head(10)

cleaned_app_version,review_rating
str,f64
null,4.076079
"""1.2.0 build""",4.211864
"""1.2.1 build""",3.819461
"""1.2.2 build""",3.67928
"""1.3.0 build""",3.689362
"""1.4.0 build""",3.036388
"""1.4.1 build""",3.344689
"""1.5.0 build""",3.676969
"""1.5.1 build""",3.417691


In [13]:
import plotly.express as px


fig = px.line(
    df,
    x="cleaned_app_version",
    y="review_rating",
    title=f"Average Review Rating of App Over Time",
)
fig.show()

In [14]:
data.head(2)

review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp,cleaned_app_version
str,str,str,str,i64,i64,str,str,str
"""7e73f80e-a8fd-…","""15261855397701…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…","""1.2.0 build"""
"""dab55eca-c2a0-…","""23438294286543…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…","""1.2.0 build"""


In [15]:
review_data: pl.DataFrame = data.select(["review_text", "review_rating"])

review_data.head()

review_text,review_rating
str,i64
"""Works great on…",5
"""Works great on…",5
"""Works great on…",5
"""Working perfec…",5
"""cm7 2.3.3 N1""",5


In [16]:
from datasets import Dataset


review_dataset: Dataset = Dataset.from_pandas(review_data.to_pandas())
review_dataset

Dataset({
    features: ['review_text', 'review_rating'],
    num_rows: 1531103
})

In [17]:
print(review_dataset[:3])

{
    'review_text': [
        'Works great on my Evo! Glad android phones are getting love now!',
        "Works great on HTC incredible. Can't wait to try video out.",
        'Works great on nexus s'
    ],
    'review_rating': [5, 5, 5]
}

In [18]:
SEED: int = 45
N: int = 250_000

sample_data: Dataset = review_dataset.shuffle(seed=23).select(range(N))
sample_data

Dataset({
    features: ['review_text', 'review_rating'],
    num_rows: 250000
})

In [19]:
def calculate_length(example: dict[str, Any]) -> dict[str, Any]:
    """This returns the length of the reviews."""
    result: dict[str, Any] = {
        "review_length": [len(x) for x in example.get("review_text")]
    }
    return result

In [20]:
sample_data = sample_data.map(calculate_length, batched=True)
print(sample_data[:3])

Map:   0%|          | 0/250000 [00:00<?, ? examples/s]

{
    'review_text': [
        'The app boots me out of Samsung Smart View every time I open it. Was working on my old Samsung S5 and s6 
but will not work in s7. Very annoying issue.',
        'I like all movies in Netflix exciting',
        'Great app'
    ],
    'review_rating': [2, 5, 5],
    'review_length': [150, 37, 9]
}

In [21]:
# Drop reviews with fewer characters
sample_data = sample_data.filter(lambda x: x.get("review_length") > 10)
sample_data

Filter:   0%|          | 0/250000 [00:00<?, ? examples/s]

Dataset({
    features: ['review_text', 'review_rating', 'review_length'],
    num_rows: 175590
})

In [22]:
save_fp: Path = Path("../../../../data/netflix_cleaned_data.jsonl")

sample_data.to_json(save_fp)

Creating json from Arrow format:   0%|          | 0/176 [00:00<?, ?ba/s]

24094158